In [0]:
from pyspark.sql import functions as F

In [0]:
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

In [0]:
origem_path = 'fomacao_microsoft_power_bi_profisional.silver.pessoas'

In [0]:
df_gold = (spark.read.table(origem_path)
                .withColumn('_ingest_ts_utc', F.current_timestamp())
                .withColumn("_ingest_date", F.to_date(F.col("_ingest_ts_utc")))
                .select(
                    'Codigo', 
                    'email',
                    'login', 
                    'dominio',
                    'NomeCompleto', 
                    'Genero', 
                    'Avatar', 
                    'Pais', 
                    'Nascimento', 
                    'AnoNascimento',
                    '_ingest_ts_utc', 
                    '_ingest_date'
                    )
                )

In [0]:
tbl = 'pessoas'
destino_path = f'fomacao_microsoft_power_bi_profisional.gold.{tbl}'

df_gold = (df_gold.write
             .format('delta')
             .mode('overwrite')
             .option('overwriteSchema', 'true')
             .partitionBy('_ingest_date')
             .saveAsTable(destino_path))

print(f'Contagem de registros: {spark.table(destino_path).count()}')